In [18]:
import pandas as pd
import numpy as no

import seaborn as sns

import re
import nltk
nltk.download(['stopwords','punkt','wordnet','omw-1.4'])
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [19]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [20]:
df = pd.read_csv("/content/drive/MyDrive/4.1/Ass 3/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


we are going to split the training and testing data set into train and validation test set


In [21]:
len(df)

20800

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.callbacks import EarlyStopping

splitting data to avoid leakages

In [23]:
x_train,x_test,y_train,y_test = train_test_split(df['text'],
                                                 df['label'],
                                                 test_size=0.2,
                                                 random_state=42)

Tokenizer

In [44]:
x_train = [str(text) for text in x_train]
x_test = [str(text) for text in x_test]

max_len = 128
truncating = 'post'
padding = 'post'
oov_tok = '<OOV>'
vocab_size = 1000

tokenizer = Tokenizer(num_words=vocab_size, char_level=False, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)

In [45]:
import pickle

# Specify the file path where you want to save the tokenizer
tokenizer_file_path = "my_token.pk1"

# Save the tokenizer to the file
with open(tokenizer_file_path, 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

In [32]:
training_sequences = tokenizer.texts_to_sequences(x_train)
training_sequences = pad_sequences(training_sequences, maxlen=max_len, padding=padding, truncating=truncating)

testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_sequences = pad_sequences(testing_sequences, maxlen=max_len, padding=padding, truncating=truncating)

checking shape of final processed data

In [33]:
print("Shape of training_sequences:", training_sequences.shape)
print("Shape of testing_sequences:", testing_sequences.shape)

Shape of training_sequences: (16640, 128)
Shape of testing_sequences: (4160, 128)


Building model

In [39]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense

embedding_dim = 32

model = Sequential()
model.add(Embedding(vocab_size,
                    embedding_dim,
                    input_length=max_len))

model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Corrected activation function name
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


fitting model using training sequence

In [40]:
from IPython.core import history
early_stop = EarlyStopping(monitor="val_loss", patience=2)
history = model.fit(training_sequences,
                    y_train,
                    epochs = 30,
                    validation_data = (testing_sequences,y_test),
                    callbacks = [early_stop],
                    verbose = 2)

Epoch 1/30
520/520 - 87s - loss: 0.5912 - accuracy: 0.6690 - val_loss: 0.5904 - val_accuracy: 0.6904 - 87s/epoch - 167ms/step
Epoch 2/30
520/520 - 90s - loss: 0.5890 - accuracy: 0.6797 - val_loss: 0.4619 - val_accuracy: 0.8197 - 90s/epoch - 172ms/step
Epoch 3/30
520/520 - 88s - loss: 0.5559 - accuracy: 0.7021 - val_loss: 0.6394 - val_accuracy: 0.6839 - 88s/epoch - 169ms/step
Epoch 4/30
520/520 - 86s - loss: 0.5873 - accuracy: 0.6978 - val_loss: 0.4048 - val_accuracy: 0.8238 - 86s/epoch - 165ms/step
Epoch 5/30
520/520 - 86s - loss: 0.2907 - accuracy: 0.8890 - val_loss: 0.2557 - val_accuracy: 0.8998 - 86s/epoch - 165ms/step
Epoch 6/30
520/520 - 86s - loss: 0.2027 - accuracy: 0.9270 - val_loss: 0.1776 - val_accuracy: 0.9339 - 86s/epoch - 165ms/step
Epoch 7/30
520/520 - 88s - loss: 0.1598 - accuracy: 0.9419 - val_loss: 0.2203 - val_accuracy: 0.9221 - 88s/epoch - 169ms/step
Epoch 8/30
520/520 - 86s - loss: 0.1508 - accuracy: 0.9454 - val_loss: 0.1674 - val_accuracy: 0.9368 - 86s/epoch - 165

In [41]:
# Save the model in HDF5 format (.h5)
model.save('my_model.h5')

# Alternatively, save the model in TensorFlow SavedModel format
model.save('my_model_saved_model')

# Download the saved model to your local machine (optional)
from google.colab import files
files.download('my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
test_news = ['Alert! Alert! the sky is definitly going to fall by the end of this yeah!!!!']
test_news_seq = tokenizer.texts_to_sequences(test_news)
test_news_seq = pad_sequences(test_news_seq,
                              maxlen = max_len,
                              padding = padding,
                              truncating = truncating)

print(test_news_seq.shape)
test_news_pred = model.predict(test_news_seq)
print(test_news_pred)

(1, 128)
1/1 [==============================] - 0s 467ms/step
[[0.97714204]]
